In [28]:
import spacy
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.neighbors import NearestNeighbors
import numpy as np

### NER extraction

In [2]:
nlp = spacy.load("en_core_web_sm")

In [11]:
doc = nlp(u"?")

In [37]:
def extract_filter_ents(text):
    doc = nlp(text)
    ents_extrd = []
    for ent in doc.ents:
        print(ent.label_)
        if ent.label_ in ["PERSON", "ORG", "NORP", "FAC", "GPE", "LOC", "PRODUCT", "EVENT", "WORK_OF_ART"]:
            ents_extrd.append(ent.text)       
    return ents_extrd
        

In [51]:
extract_filter_ents("Winged Victory of Samothrace by Boticelli")

ORG


['Winged Victory of Samothrace']

### Nearest neighbours of the vector

In [33]:
def return_nn(matrix, vector, nn):
    nbrs = NearestNeighbors(nn)
    nbrs.fit(matrix)
    indices = nbrs.kneighbors(vector, return_distance=False)
    indices = indices[0]
#     print(matrix[tree.query(list(vector))[2]])
    
    

In [34]:
x = np.random.rand(1000, 200)
y = np.random.rand(1, 200)
print(y.shape)
return_nn(x, y, 10)

(1, 200)
[387 364 314 679 752 573  12 450 774 829]


### Zero shot model

In [87]:
class ZS_net(nn.Module):
        def __init__(self, question_emb_size):
            super(ZS_net, self).__init__()
            self.W = nn.Parameter(torch.nn.init.xavier_uniform_(torch.Tensor(question_emb_size, question_emb_size)))
            
        
        def forward(self, question_emb, label_emb):
            mult = torch.mm(question_emb, self.W)
            output = torch.sigmoid(torch.mm(mult, label_emb.t()))
            return output


In [105]:
question_emb = torch.rand(1, 512)
label_emb = torch.rand(1, 512)
question1_emb = torch.rand(1, 512)
label1_emb = torch.rand(1, 512)

In [89]:
model = ZS_net(512)

In [90]:
model(question_emb, label_emb)

tensor([[0.9999]], grad_fn=<SigmoidBackward>)

In [95]:
optimizer = torch.optim.Adam(model.parameters())

AttributeError: module 'torch.nn' has no attribute 'MSE'

In [111]:
data_points = [(question_emb, label_emb, 1), (question1_emb, label1_emb, 0)]
num_batches = 1
num_epochs = 10

In [112]:
for i in range(num_epochs):
    for j in range(num_batches):
        input_question = data_points[0][0]
        input_label = data_points[0][1]
        y = torch.tensor(data_points[0][2]).float()
        
        output = model(input_question, input_label)
        optimizer.zero_grad()
        loss = F.mse_loss(output, y)
        print(loss)
        loss.backward()
        optimizer.step()
    input_question = data_points[1][0]
    input_label = data_points[1][1]
    y = torch.tensor(data_points[1][2]).float()
    print(F.mse_loss(output, y))

tensor(0., grad_fn=<MseLossBackward>)
tensor(1., grad_fn=<MseLossBackward>)
tensor(0., grad_fn=<MseLossBackward>)
tensor(1., grad_fn=<MseLossBackward>)
tensor(0., grad_fn=<MseLossBackward>)
tensor(1., grad_fn=<MseLossBackward>)
tensor(0., grad_fn=<MseLossBackward>)
tensor(1., grad_fn=<MseLossBackward>)
tensor(0., grad_fn=<MseLossBackward>)
tensor(1., grad_fn=<MseLossBackward>)
tensor(0., grad_fn=<MseLossBackward>)
tensor(1., grad_fn=<MseLossBackward>)
tensor(0., grad_fn=<MseLossBackward>)
tensor(1., grad_fn=<MseLossBackward>)
tensor(0., grad_fn=<MseLossBackward>)
tensor(1., grad_fn=<MseLossBackward>)
tensor(0., grad_fn=<MseLossBackward>)
tensor(1., grad_fn=<MseLossBackward>)
tensor(0., grad_fn=<MseLossBackward>)
tensor(1., grad_fn=<MseLossBackward>)
